In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import segmentation_models_pytorch as smp
from segmentation_models_pytorch.encoders import get_preprocessing_fn

In [ ]:
import os
import wandb
#os.environ['WANDB_MODE'] = 'dryrun'
wandb.init("sky-eye-full")
conf = wandb.config

In [ ]:
from pprint import pprint
pprint(dict(conf))

In [ ]:
from glob import glob
train_dir = '/home/jupyter/datasets/xview/train'
test_dir = '/home/jupyter/datasets/xview/test'

In [ ]:
from tqdm import tqdm_notebook as tqdm
import numpy as np
import torch
from torch import nn
from xv.util import vis_im_mask
import segmentation_models_pytorch as smp

In [ ]:
import albumentations as al

augment = al.Compose([
        al.HorizontalFlip(p=conf.aug_prob),
        al.VerticalFlip(p=conf.aug_prob),
        al.RandomRotate90(p=conf.aug_prob),
        al.Transpose(p=conf.aug_prob),
        al.GridDistortion(p=conf.aug_prob, distort_limit=.2),
        al.ShiftScaleRotate(p=conf.aug_prob),
        al.RandomBrightnessContrast(p=conf.aug_prob)
])

In [ ]:
from xv.nn.solaris.model_io import get_model
from xv.nn.nets import DownscaleLayer, XVNet
import segmentation_models_pytorch as smp

segmentation_types = {
    'PSPNet': smp.PSPNet,
    'FPN': smp.FPN,
    'Linknet': smp.Linknet,
    'Unet': smp.Unet
}

if 'pretrained_model' in dict(conf):
    model = get_model(conf.pretrained_model, 'torch', pretrained=True)
    preprocess_fn = None#get_preprocessing_fn('efficientnet-b0') #Imagenet 
else:
    model = segmentation_types[conf.segmentation_arch](conf.encoder,
                                                       classes=1,
                                                       activation='sigmoid')
    preprocess_fn = get_preprocessing_fn(conf.encoder)

model = model.cuda()

In [ ]:
import random
from xv import dataset

random.seed(hash("😂"))


all_files = glob(f'{train_dir}/labels/*pre_disaster.json')
random.shuffle(all_files)

dev_ix = int(len(all_files)*.20)
dev_files = all_files[:dev_ix]
train_files = all_files[dev_ix:]

train_instances, dev_instances = dataset.get_instances(train_files), dataset.get_instances(dev_files)

len(train_instances), len(dev_instances)

In [ ]:
train_dataset = dataset.BuildingSegmentationDataset(
    instances=train_instances,
    resolution=conf.training_resolution,
    augment=augment,
    preprocess_fn=preprocess_fn,
)

dev_dataset = dataset.BuildingSegmentationDataset(
    instances=dev_instances,
    resolution=conf.training_resolution,
    augment=None,
    preprocess_fn=preprocess_fn,
)

train_loader = torch.utils.data.DataLoader(
    train_dataset,
    batch_size=conf.batch_size,
    shuffle=True,
    num_workers=10,
)

dev_loader = torch.utils.data.DataLoader(
    dev_dataset,
    batch_size=conf.batch_size,
    shuffle=True,
    num_workers=10,
)

In [ ]:
from xv.nn.losses import loss_dict, WeightedLoss
loss = WeightedLoss({loss_dict[l]():w for l, w in conf.loss_weights.items()})

In [ ]:
import apex

optims = {
    'adam': torch.optim.Adam
}

optim = optims[conf.optim](model.parameters(), lr=conf.lr)

In [ ]:
from apex import amp
model, optim = amp.initialize(model, optim, opt_level=conf.amp_opt_level);

In [ ]:
wandb.watch(model);

In [ ]:
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optim, factor=conf.scheduler_factor, patience=conf.scheduler_patience)

In [ ]:
best_score = 0
epoch = 0

In [ ]:
from xv import run
for epoch in range(epoch, conf.epochs):
    metrics = {'epoch': epoch}
    train_metrics = run.train_segment(model, optim, train_loader, loss)
    metrics.update(train_metrics)
    
    dev_metrics = run.evaluate_segment(model, dev_loader, loss)
    metrics.update(dev_metrics)
    
    examples = run.sample_masks(model, dev_instances, preprocess_fn)
    metrics['examples'] = [wandb.Image(e) for e in examples]
    
    wandb.log(metrics)
    
    scheduler.step(metrics['loss'])
    
    score = metrics['building:f1']
    
    if score > best_score:
        torch.save(model.state_dict(), os.path.join(wandb.run.dir, "state_dict.pth"))
        best_score = score

In [ ]:
ix = 1000
i = train_dataset[ix]
images, masks = i['images'], i['masks']
image = images['post']
image = np.array(train_dataset.inverse_transform_image(image))
util.vis_im_mask(image, masks['damage'], size=(512*2, 512*2), opacity=.3);

In [ ]:
from collections import Counter
counts = Counter(len(i['pre']['features']) for i in train_dataset.instances)

In [ ]:
for i in run.sample_masks(model, dev_instances, preprocess_fn, sz=1024):
    display(i)

In [34]:
from PIL import Image
for i in random.sample(dev_instances, 1):
    with torch.no_grad():
        img = np.array(Image.open(i['file_name']))
        model_in = preprocess_fn(img).transpose(2,0,1)
        model_in = torch.tensor(model_in)
        model_in = model_in.reshape(1, *model_in.shape)
        mask = model(model_in.cuda())
        mask = np.array((mask > 0).cpu())

In [44]:
polys = Mask(mask.reshape((1024,1024))).polygons()

In [52]:
from shapely import 

<module 'shapely.wkt' from '/opt/anaconda3/lib/python3.7/site-packages/shapely/wkt.py'>

In [28]:
from imantics import Mask